In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summary-data-all-columns/Summary_data_all_columns.csv


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup, GPT2ForSequenceClassification, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm.auto import tqdm

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup, GPT2ForSequenceClassification, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch

df = pd.read_csv('/kaggle/input/summary-data-all-columns/Summary_data_all_columns.csv')
df2 = df[['GPT-2 Summarization', 'engine_type', 'drive_type']]
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', max_length=1024)

In [ ]:
# Define padding function
def pad_sequence(seq, max_len):
    seq_len = len(seq)
    if seq_len < max_len:
        seq = seq + [0] * (max_len - seq_len)
    elif seq_len > max_len:
        seq = seq[:max_len]
    return seq

In [ ]:
drive_type_labels = {'four wheel drive  ': 0, 'rear wheel drive  ': 1, 'front wheel drive  ': 2, 'all wheel drive  ': 3}
engine_type_labels = {'Inline 4 ': 0, 'V6 ': 1, 'V8 ': 2, 'Inline 6 ': 3, 'V12 ': 4, 'Inline 5 ': 5, 'Flat 4 ': 6, 'Inline 3 ': 7, 'V10 ': 8}

df2['drive_type_labels'] = df2['drive_type'].map(drive_type_labels)
df2['engine_type_labels'] = df2['engine_type'].map(engine_type_labels)

#tokenized_text = tokenizer(df2['GPT-2 Summarization'].tolist(), truncation=True, padding=True, max_length=1024, return_tensors='pt')
tokenized_text = df2['GPT-2 Summarization'].apply((lambda x: tokenizer.encode(x, max_length=1024,truncation=True)))

In [ ]:
lii = df2['engine_type_labels'].tolist()
labels = torch.LongTensor(lii)

In [ ]:
max_len = 1024
padded_data = [pad_sequence(seq, max_len) for seq in tokenized_text.values]
# Convert integers to strings and pad with spaces
padded_strings = [[' ' + chr(i) for i in seq] for seq in padded_data]
padded_strings = [[s[1:] for s in seq] for seq in padded_strings]
padded_strings = [pad_sequence(seq, max_len) for seq in padded_strings]
# Vectorize padded data
vectorized_data = np.vectorize(lambda x: ord(x))(padded_strings)

In [9]:
# Flatten padded_data
padded_data = np.array([np.ravel(x) for x in padded_data]).astype(np.int64)
# Create attention masks
attention_masks = np.where(padded_data != 0, 1, 0)

In [10]:
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=9)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
train_dataset = torch.utils.data.TensorDataset(torch.tensor(padded_data), torch.tensor(attention_masks), labels)

batch_size = 2
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)*10)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
print(device)
model.train()

cuda


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid

In [13]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

# set padding token id to 0
tokenizer.pad_token_id = 0
model.config.pad_token_id = model.config.eos_token_id

In [14]:
epochs = 3

for epoch in tqdm(range(1, epochs+1)):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader):
        inputs = batch[0].to(device)
        masks = batch[1].to(device)
        labels = batch[2].to(device)
        model.train()
        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        acc = torch.mean((torch.argmax(outputs.logits, dim=1) == labels).float())
        
    
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1571 [00:00<?, ?it/s]

Average train loss: 1.3353396226254184


  0%|          | 0/1571 [00:00<?, ?it/s]

Average train loss: 0.7291442123090554


  0%|          | 0/1571 [00:00<?, ?it/s]

Average train loss: 0.5034420608865503


In [15]:
model.save_pretrained('PreFineTune_engine_type')

In [16]:
import shutil
shutil.make_archive("PreFineTune_engine_type", 'zip', "/kaggle/working/PreFineTune_engine_type")

'/kaggle/working/PreFineTune_engine_type.zip'

In [17]:
%cd /kaggle/working

/kaggle/working


In [18]:
from IPython.display import FileLink

In [19]:
 FileLink(r'PreFineTune_engine_type.zip')

/kaggle/working/PreFineTune_engine_type.zip